In [1]:
import os.path
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy import wcs
from astropy.nddata import Cutout2D
from astropy import coordinates
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import matplotlib.gridspec as gridspec
import aplpy

In [2]:
#os.system('sex -d > sextractor/goods.conf')
path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table.fits'
big_table = fits.open(path)[1].data
visual = big_table.field('Visual_Flag')
usable_b = big_table.field('GOODS_b_Usable')
usable_v = big_table.field('GOODS_v_Usable')
usable_i = big_table.field('GOODS_i_Usable')
usable_z = big_table.field('GOODS_z_Usable')
usable_105 = big_table.field('CANDELS_105_Usable')
usable_125 = big_table.field('CANDELS_125_Usable')
usable_160 = big_table.field('CANDELS_160_Usable')
usable_K = big_table.field('HUGS_Usable')
usable_36 = np.ones(len(visual))
usable_45 = np.ones(len(visual))
usable = [usable_b, usable_v, usable_i, usable_z, usable_105, usable_125, usable_160, usable_K, usable_36, usable_45]
aperture = big_table.field('Aperture') #arcsec diameter
minarea = big_table.field('Min_Area') #pixel
spitzer = big_table.field('Spitzer_Aper_Flag') #arcsec diameter


a = ['goods_b', 'goods_v', 'goods_i', 'goods_z', 'candels_105_', 'candels_125_', 'candels_160_', 'hugs_', 'spitzer_ch1_', 'spitzer_ch2_']
b = ['goods_b', 'goods_v', 'goods_i', 'goods_z', 'candels_105', 'candels_125', 'candels_160', 'hugs', 'spitzer_ch1', 'spitzer_ch2']
c = ['goods', 'goods', 'goods', 'goods', 'candels', 'candels', 'candels', 'hugs', 'spitzer', 'spitzer']
aper_pixel = 1.
min_area = 1.
for j in range(len(a)):
    for i in range(big_table.size):
        if ((visual[i] > 3) & (usable[j][i] > 0)) | ((i==716) & (j==6)):
            if j < 4:
                aper_pixel = aperture[i] / 0.03
                min_area = minarea[i]
                n_thresh = 64.
                min_cont = 0.0001
            elif j < 7:
                aper_pixel = aperture[i] / 0.06
                min_area = minarea[i]
                n_thresh = 64.
                min_cont = 0.0001
                if i == 352:
                    n_thresh = 32.
                    min_cont = 0.001
            elif j == 7:
                aper_pixel = aperture[i] * 9.4
                min_area = minarea[i]/2.
                n_thresh = 64.
                min_cont = 0.0001
            else:
                if spitzer[i] > 10.:
                    aper_pixel = 0.
                else:
                    aper_pixel = spitzer[i] *1.7
                min_area = 1.
                n_thresh = 64.
                min_cont = 0.0001
            if aper_pixel != 0.:
                conf_path = ' /Users/justin/Documents/Master_Thesis/data/catalogues/' + c[j] + '/' + b[j] + '.conf'
                im_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + '.fits'
                dualim_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + '.fits,/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + '.fits'
                cat_path = ' /Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/' + c[j] + '/cat_' + a[j] + str(i+1) + '.cat'
                wht_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + 'wht.fits'
                rms_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + 'rms.fits'
                rmswht_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + 'rms.fits,/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + 'wht.fits'
                check_path = ' /Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/' + c[j] + '/cat_' + a[j] + str(i+1) + '_aper.fits,/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/' + c[j] + '/cat_' + a[j] + str(i+1) + '_bkg.fits,/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/' + c[j] + '/cat_' + a[j] + str(i+1) + '_bkg_rms.fits' 
                if j < 4:
                    path = 'sex -c' + conf_path + im_path + ' -CATALOG_NAME' + cat_path + ' -WEIGHT_TYPE MAP_RMS -WEIGHT_GAIN N -WEIGHT_IMAGE' + rms_path + ' -DETECT_MINAREA ' + str(min_area) + ' -PHOT_APERTURES ' + str(aper_pixel) + ' -CHECKIMAGE_NAME' + check_path + ' -BACK_SIZE 90 -DEBLEND_MINCONT ' + str(min_cont) + ' -DEBLEND_NTHRESH ' + str(n_thresh) + ' -MASK_TYPE NONE'
                elif j < 7:
                    path = 'sex -c' + conf_path + im_path + ' -CATALOG_NAME' + cat_path + ' -WEIGHT_TYPE MAP_RMS -WEIGHT_GAIN N -WEIGHT_IMAGE' + rms_path + ' -DETECT_MINAREA ' + str(min_area) + ' -PHOT_APERTURES ' + str(aper_pixel) + ' -CHECKIMAGE_NAME' + check_path + ' -BACK_SIZE 60 -DEBLEND_MINCONT ' + str(min_cont) + ' -DEBLEND_NTHRESH ' + str(n_thresh) + ' -MASK_TYPE NONE'
                elif j == 7:
                    path = 'sex -c' + conf_path + im_path + ' -CATALOG_NAME' + cat_path + ' -WEIGHT_TYPE MAP_RMS -WEIGHT_GAIN N -WEIGHT_IMAGE' + rms_path + ' -DETECT_MINAREA ' + str(min_area) + ' -PHOT_APERTURES ' + str(aper_pixel) + ' -CHECKIMAGE_NAME' + check_path + ' -BACK_SIZE 50 -DEBLEND_MINCONT ' + str(min_cont) + ' -DEBLEND_NTHRESH ' + str(n_thresh) + ' -MASK_TYPE NONE'
                else:
                    path = 'sex -c' + conf_path + dualim_path + ' -CATALOG_NAME' + cat_path + ' -WEIGHT_TYPE MAP_RMS,MAP_WEIGHT -WEIGHT_GAIN N,Y -WEIGHT_IMAGE' + rmswht_path + ' -DETECT_MINAREA ' + str(min_area) + ' -PHOT_APERTURES ' + str(aper_pixel) + ' -CHECKIMAGE_NAME' + check_path + ' -BACK_SIZE 17 -DEBLEND_MINCONT ' + str(min_cont) + ' -DEBLEND_NTHRESH ' + str(n_thresh) + ' -MASK_TYPE NONE -FILTER N'
                os.system(path)
        if (visual[i] > 3) & (usable[j][i] == 0):
            print('Problem at source' + str(i+1) + ', band' + b[j])


Problem at source638, bandgoods_b
Problem at source101, bandcandels_105
Problem at source150, bandcandels_105
Problem at source195, bandcandels_105
Problem at source354, bandcandels_105
Problem at source392, bandcandels_105
Problem at source411, bandcandels_105
Problem at source453, bandcandels_105
Problem at source525, bandcandels_105
Problem at source527, bandcandels_105
Problem at source617, bandcandels_105
Problem at source638, bandcandels_105
Problem at source662, bandcandels_105
Problem at source715, bandcandels_105
Problem at source815, bandcandels_105
Problem at source817, bandcandels_105
Problem at source854, bandcandels_105
Problem at source859, bandcandels_105
Problem at source101, bandcandels_125
Problem at source717, bandcandels_125
Problem at source101, bandcandels_160
Problem at source717, bandcandels_160
